In [1]:
import os
import pandas as pd
from collections import Counter
import pymorphy3
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import to_rgb
from wordcloud import WordCloud
import numpy as np

morph = pymorphy3.MorphAnalyzer(lang='ru')

3. Найдем абсолютные и относительные частоты для каждого цвета и всех цветообозначений (по автору)

In [2]:
#Нужные списки (повторяю, так как новая тетрадка)
COLOR_WORDS = {
    'красный', 'оранжевый', 'жёлтый', 'зелёный', 'синий', 'голубой',
    'фиолетовый','чёрный', 'белый', 'серый', 'розовый', 'коричневый'}

authors = ['Пелевин', 'Сорокин', 'Иванов', 'Елизаров', 'Яхина', 'Прилепин', 'Водолазкин', 'Сальников']

In [3]:
def extract_adjectives(text):
    """Извлекает все прилагательные в начальной форме."""
    words = text.split()
    adjectives = []
    for word in words:
        parsed = morph.parse(word)[0]
        if 'ADJF' in parsed.tag:  #полное прилагательное
            adjectives.append(parsed.normal_form)
    return adjectives

In [ ]:
#Посчитаем относительные и абсолютные частоты для каждого автора (ИИ помог с оформление датафрейма и составлением топа цветов)
# Основной цикл по авторам
for author in authors:
    #Читаем список цветов
    with open(f"{author}_colors_only.txt", "r", encoding="utf-8") as f:
        colors = f.read().splitlines()
    
    #Читаем очищенный текст
    with open(f"{author}_cleaned.txt", "r", encoding="utf-8") as f:
        cleaned_text = f.read()

    #Считаем абсолютные частоты
    color_freq = Counter(colors)  # частота каждого цвета
    total_color_count = sum(color_freq.values())  # общее число цветообозначений

    # Общее число слов в тексте
    total_words = len(cleaned_text.split())

    # Извлекаем все прилагательные
    adjectives = extract_adjectives(cleaned_text)
    total_adjectives = len(adjectives)

    #Считаем относительные частоты
    results = []
    for color, abs_freq in color_freq.items():
        # Относительная частота: цвет / все цвета (доля данного цвета среди всех цветообозначений)
        rel_freq_color_total = abs_freq / total_color_count 
        # Относительная частота: цвет / все слова в объединенном тексте автора
        rel_freq_words = abs_freq / total_words
        # Относительная частота: цвет / все прилагательные в объединенном тексте автора
        rel_freq_adj = abs_freq / total_adjectives
        # Число вхождений цвета на 1000 слов (т.е. нормализуем абсолютную частоту)
        norm_freq = (abs_freq * 1000) / total_words

        results.append({
            'автор': author,
            'цвет': color,
            'абс_частота_цвета': abs_freq,
            'нормализ_частота_на_1000': round(norm_freq, 4),
            'отн_частота_цвет_все_цвета': rel_freq_color_total,
            'отн_частота_цвет_все_слова': rel_freq_words,
            'отн_частота_цвет_все_прил': rel_freq_adj
        })

    #Рассчитываем относительные частоты для "всех цветов" суммарно
    total_rel_freq_words = total_color_count / total_words #относительная частота: все цвета / все слова в объедненном тексте по автору
    total_rel_freq_adj = total_color_count / total_adjectives #относительная частота: все цвета / все прилагательные в объединенном тексте по автору

    results.append({
        'автор': author,
        'цвет': 'все_цвета',
        'абс_частота_цвета': total_color_count,
        'отн_частота_цвет_все_слова': total_rel_freq_words,
        'отн_частота_цвет_все_прил': total_rel_freq_adj
    })

#Создаём датафрейм (результаты по частотам для каждого автора отсортированы в порядке убывания)
    df = pd.DataFrame(results)

    #Сортируем: сначала «все_цвета», затем остальные цвета по убыванию абс. частоты:
    #отделяем строку «все_цвета»
    all_colors_row = df[df['цвет'] == 'все_цвета'].copy()
    other_rows = df[df['цвет'] != 'все_цвета'].copy()

    #сортируем остальные строки по убыванию 'абс_частота_цвета'
    other_rows_sorted = other_rows.sort_values(
        by='абс_частота_цвета', ascending=False
    ).reset_index(drop=True)

    #объединяем: сначала «все_цвета», потом отсортированные цвета
    df_sorted = pd.concat([all_colors_row, other_rows_sorted], ignore_index=True)

    #присваиваем индекс от 0 до N-1
    df_sorted.index = range(len(df_sorted))

    #корректируем столбец 'автор'
    df_sorted['автор'] = author
    df_sorted.loc[1:, 'автор'] = '  '

    #сохраняем в CSV
    df_sorted.to_csv(
        f"{author}_color_frequencies.csv",
        index=False,
        encoding='utf-8-sig'
    )
    #выводим для контроля
    print(f"\nРезультаты для автора: {author}")
    print(df_sorted)


Результаты для автора: Пелевин
      автор        цвет  абс_частота_цвета  нормализ_частота_на_1000  \
0   Пелевин   все_цвета               2636                       NaN   
1                чёрный                760                    0.8744   
2               красный                535                    0.6155   
3                 белый                289                    0.3325   
4               зелёный                257                    0.2957   
5                жёлтый                224                    0.2577   
6                 синий                172                    0.1979   
7                 серый                135                    0.1553   
8               розовый                 78                    0.0897   
9               голубой                 70                    0.0805   
10            оранжевый                 47                    0.0541   
11           коричневый                 42                    0.0483   
12           фиолетовый         

4. Посчитаем частоты для сочетаний "цветообозначение + существительное"

In [4]:
all_pairs_lemma = []      # лемматизированные сочетания «цвет + сущ»
all_colors_lemma = []     # леммы цветообозначений
all_nouns_lemma = []      # леммы существительных
author_data = {}          # данные по авторам

#Указываем путь снова (так как тетрадка новая)
path = r"C:\Users\1610043\Desktop\Проектик"

#Обрабатываем каждого автора из готового списка
for author in authors:
    filepath = os.path.join(path, f"{author}_color_noun_pairs.txt")

    pairs_lemma = []
    colors_lemma = []
    nouns_lemma = []

    with open(filepath, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            # Разбираем строку: предполагаем порядок «цвет существительное» (так как сохраняли пары в файл таким образом)
            parts = line.split()
            color_orig = parts[0]
            noun_orig = parts[1]
            # Лемматизация цвета
            color_parsed = morph.parse(color_orig)[0]
            color_lemma = color_parsed.normal_form
            # Лемматизация существительного
            noun_parsed = morph.parse(noun_orig)[0]
            noun_lemma = noun_parsed.normal_form
            # Формируем лемматизированное сочетание
            pair_lemma = f"{color_lemma} {noun_lemma}"

            pairs_lemma.append(pair_lemma)
            colors_lemma.append(color_lemma)
            nouns_lemma.append(noun_lemma)

    # Накапливаем для общих топов
    all_pairs_lemma.extend(pairs_lemma)
    all_colors_lemma.extend(colors_lemma)
    all_nouns_lemma.extend(nouns_lemma)

    # Сохраняем для автора
    author_data[author] = {
        'pairs': pairs_lemma,
        'colors': colors_lemma,
        'nouns': nouns_lemma
    }

#Строим общие топы
# Топ-10 сочетаний цвет + сущ
print("Топ‑10 сочетаний «цвет + существительное»  (все авторы)")
print("-" * 60)
top_pairs_all = Counter(all_pairs_lemma).most_common(10)
for i, (pair, freq) in enumerate(top_pairs_all, 1):
    print(f"{i}. {pair} — {freq} раз")

#Топ-5 цветов в составе сочетаний
print("\n"+"Топ‑5 цветов в сочетаниях «цвет + существительное» (все авторы)")
print("-" * 60)
top_colors_all = Counter(all_colors_lemma).most_common(5)
for i, (color, freq) in enumerate(top_colors_all, 1):
    print(f"{i}. {color} — {freq} раз")
    
#Топ-5 существительных в составе сочетаний
print("\n" + "Топ‑5 существительных в сочетаниях «цвет + существительное» (все авторы)")
print("-" * 60)
top_nouns_all = Counter(all_nouns_lemma).most_common(5)
for i, (noun, freq) in enumerate(top_nouns_all, 1):
    print(f"{i}. {noun} — {freq} раз")

#Индивидуальные топы по авторам
print("\n" + "Индивидуальные топы по авторам")
print("-" * 60)

for author in authors:
    print(f"\n--- {author} ---")
    data = author_data[author]

    # Топ‑10 сочетаний цвет + сущ
    top_pairs = Counter(data['pairs']).most_common(10)
    print("Топ‑10 сочетаний «цвет + существительное»:")
    for i, (pair, freq) in enumerate(top_pairs, 1):
        print(f"  {i}. {pair} — {freq} раз")

    # Топ‑5 цветов в сочетаниях
    top_colors = Counter(data['colors']).most_common(5)
    print("Топ‑5 цветов в сочетаниях «цвет + существительное»:")
    for i, (color, freq) in enumerate(top_colors, 1):
        print(f"  {i}. {color} — {freq} раз")

    # Топ‑5 существительных в сочетаниях
    top_nouns = Counter(data['nouns']).most_common(5)
    print("Топ‑5 существительных в сочетаниях «цвет + существительное»:")
    for i, (noun, freq) in enumerate(top_nouns, 1):
        print(f"  {i}. {noun} — {freq} раз")

#Сохраняем в CSV
# Общие топы
pd.DataFrame(top_pairs_all, columns=["Сочетание", "Частота"]).to_csv(
    "топ_сочетаний_все_авторы_леммы.csv", index=False, encoding="utf-8-sig"
)
pd.DataFrame(top_colors_all, columns=["Цвет", "Частота"]).to_csv(
    "топ_цветов_все_авторы_леммы.csv", index=False, encoding="utf-8-sig"
)
pd.DataFrame(top_nouns_all, columns=["Существительное", "Частота"]).to_csv(
    "топ_сущ_все_авторы_леммы.csv", index=False, encoding="utf-8-sig"
)
# Индивидуальные топы
for author in authors:
    data = author_data[author]
    pd.DataFrame(Counter(data['pairs']).most_common(10), columns=["Сочетание", "Частота"]).to_csv(
        f"топ_сочетаний_{author}_леммы.csv", index=False, encoding="utf-8-sig"
    )
    pd.DataFrame(Counter(data['colors']).most_common(5), columns=["Цвет", "Частота"]).to_csv(
        f"топ_цветов_{author}_леммы.csv", index=False, encoding="utf-8-sig"
    )
    pd.DataFrame(Counter(data['nouns']).most_common(5), columns=["Существительное", "Частота"]).to_csv(
        f"топ_сущ_{author}_леммы.csv", index=False, encoding="utf-8-sig"
    )


Топ‑10 сочетаний «цвет + существительное»  (все авторы)
------------------------------------------------------------
1. красный жидкость — 63 раз
2. белый свет — 63 раз
3. чёрный смерть — 35 раз
4. красный церемония — 32 раз
5. белый халат — 31 раз
6. чёрный вода — 29 раз
7. чёрный дыра — 26 раз
8. жёлтый господин — 26 раз
9. красный армия — 24 раз
10. чёрный небо — 21 раз

Топ‑5 цветов в сочетаниях «цвет + существительное» (все авторы)
------------------------------------------------------------
1. чёрный — 1385 раз
2. белый — 1190 раз
3. красный — 846 раз
4. синий — 424 раз
5. зелёный — 342 раз

Топ‑5 существительных в сочетаниях «цвет + существительное» (все авторы)
------------------------------------------------------------
1. свет — 142 раз
2. цвет — 114 раз
3. жидкость — 69 раз
4. небо — 57 раз
5. халат — 49 раз

Индивидуальные топы по авторам
------------------------------------------------------------

--- Пелевин ---
Топ‑10 сочетаний «цвет + существительное»:
  1. красный жид

5.Визуализация (1.Цветообозначения по авторам
2. Словосочетания цвет + сущ)

In [5]:
# Загружаем и корректируем данные всех авторов
dataframes = []

for author in authors:
    filename = f"{author}_color_frequencies.csv"
    df = pd.read_csv(filename, encoding='utf-8-sig')
    # Извлекаем фамилию автора из строки с «все_цвета»
    all_colors_row = df[df['цвет'] == 'все_цвета']
    author_name = all_colors_row['автор'].iloc[0]
    df['автор'] = author_name
    dataframes.append(df)

# Объединяем все данные
df_all = pd.concat(dataframes, ignore_index=True)
# Убираем строки с «все_цвета»
df_all = df_all[df_all['цвет'] != 'все_цвета']

#Проверяем авторов
print("Уникальные авторы в данных:", df_all['автор'].unique())

Уникальные авторы в данных: ['Пелевин' 'Сорокин' 'Иванов' 'Елизаров' 'Яхина' 'Прилепин' 'Водолазкин'
 'Сальников']


5.1.1. Сделаем визуализацию значений абс. частоты на 1000 слов для каждого автора

In [6]:
#ИИ помог с графиком (добавление цветов + подписать значения)
def plot_author_colors(df, author):
    """ Создаёт столбчатую диаграмму абс. частотности цветообозначений на 1000 слов для указанного автора на основе данных из датафрейма"""
    author_data = df[df['автор'] == author].copy()
    # Сортируем по нормализованной частоте (на 1000 слов)
    author_data = author_data.sort_values('нормализ_частота_на_1000', ascending=False).head(12)

    # Словарь соответствия: русское название цвета → код цвета matplotlib
    color_mapping = {
        'красный': 'red',
        'оранжевый': 'orange',
        'жёлтый': 'yellow',
        'зелёный': 'green',
        'синий': 'blue',
        'голубой': 'lightblue',
        'фиолетовый': 'purple',
        'чёрный': 'black',
        'белый': 'white',
        'серый': 'gray',
        'розовый': 'pink',
        'коричневый': 'brown'
    }

    # Формируем список цветов для столбцов
    color_list = [color_mapping[color_name] for color_name in author_data['цвет']]

    plt.figure(figsize=(12, 6))
    # Создаём столбчатую диаграмму
    bars = plt.bar(
        author_data['цвет'],
        author_data['нормализ_частота_на_1000'],
        color=color_list
    )

    # Добавляем обводку для белых столбцов (чтобы были видны на белом фоне)
    for bar, color_name in zip(bars, author_data['цвет']):
        if color_name == 'белый':
            bar.set_edgecolor('black')
            bar.set_linewidth(2)

    # Добавляем значения над столбцами
    for bar, value in zip(bars, author_data['нормализ_частота_на_1000']):
        plt.text(
            bar.get_x() + bar.get_width() / 2,
            bar.get_height() + max(author_data['нормализ_частота_на_1000']) * 0.01,
            f'{value}',
            ha='center',
            va='bottom',
            fontsize=9,
            color='black'
        )

    plt.xlabel('Цветообозначение')
    plt.ylabel('Абсолютная частота на 1000 слов')
    plt.title(f'{author}')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()

    filename = f"visualization_{author}_color_frequencies.png"
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    print(f"Сохранён файл: {filename}") #проверяем, что сохранилось
    plt.close()

# Строим графики для всех авторов
for author in authors:
    plot_author_colors(df_all, author)

Сохранён файл: visualization_Пелевин_color_frequencies.png
Сохранён файл: visualization_Сорокин_color_frequencies.png
Сохранён файл: visualization_Иванов_color_frequencies.png
Сохранён файл: visualization_Елизаров_color_frequencies.png
Сохранён файл: visualization_Яхина_color_frequencies.png
Сохранён файл: visualization_Прилепин_color_frequencies.png
Сохранён файл: visualization_Водолазкин_color_frequencies.png
Сохранён файл: visualization_Сальников_color_frequencies.png


5.1.2. Сделаем общий график авторов (для сравнения значений абсолютной частоты на 1000 слов)

In [7]:
#ИИ о-о-чень помог с графиком
def plot_stacked_horizontal_colors_with_precomputed_ratios(df, authors):
    """ Создаёт горизонтальный график с отн. частотами каждого цвета на все цветообозначения у автора (в долях)"""
    color_mapping = {
        'красный': 'red',
        'оранжевый': 'orange',
        'жёлтый': 'yellow',
        'зелёный': 'green',
        'синий': 'blue',
        'голубой': 'lightblue',
        'фиолетовый': 'purple',
        'чёрный': 'black',
        'белый': 'white',
        'серый': 'gray',
        'розовый': 'pink',
        'коричневый': 'brown'
    }

    #Используем готовые доли из колонки 'отн_частота_цвет_все_цвета'
    author_data_list = []
    for author in authors:
        author_df = df[df['автор'] == author].copy()
        author_df['доля'] = author_df['отн_частота_цвет_все_цвета']
        author_data_list.append(author_df)

    fig, ax = plt.subplots(figsize=(12, 8))

    y_positions = np.arange(len(authors))
    bar_height = 0.6
    current_left = np.zeros(len(authors))

    for color_name, color_code in color_mapping.items():
        color_values = []
        for i, author in enumerate(authors):
            author_data = author_data_list[i]
            color_row = author_data[author_data['цвет'] == color_name]
            if not color_row.empty:
                color_values.append(color_row['доля'].iloc[0])
            else:
                color_values.append(0)
        if color_name == 'белый':
            edge_color = 'black'
            line_width = 2
        else:
            edge_color = 'white'
            line_width = 1

        ax.barh(
            y_positions,
            color_values,
            height=bar_height,
            left=current_left,
            color=color_code,
            edgecolor=edge_color,
            linewidth=line_width
        )
        current_left += np.array(color_values)

    ax.set_yticks(y_positions)
    ax.set_yticklabels(authors)
    ax.set_xlabel('Доля цветовой палитры автора')
    ax.set_title('Распределение цветообозначений по авторам\n(относит. частота цвета/кол-во всеъ цве-й у автора)')

    ax.grid(axis='x', alpha=0.3)

    existing_colors = df[df['автор'].isin(authors)]['цвет'].unique()
    legend_elements = [
        plt.Rectangle((0, 0), 1, 1, color=color_mapping[c], label=c)
        for c in existing_colors if c in color_mapping
    ]
    ax.legend(handles=legend_elements, bbox_to_anchor=(1.05, 1), loc='upper left')

    plt.tight_layout()

    filename = "color_distribution_ratios.png"
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    print(f"Сохранён файл: {filename}") #Проверка
    plt.close()

plot_stacked_horizontal_colors_with_precomputed_ratios(df_all, authors)

Сохранён файл: color_distribution_ratios.png


5.1.3 Сделаем тепловую карту для визуализации доли каждого цвета от числа прилагательных (на каждого автора)

In [8]:
heatmap_data = df_all.pivot_table(
    index='цвет',
    columns='автор',
    values='отн_частота_цвет_все_прил',
    fill_value=0
) * 100  # в процентах

plt.figure(figsize=(10, 8))
sns.heatmap(
    heatmap_data,
    annot=True,
    fmt='.1f', # один знак после запятой
    cmap='Blues',
    cbar_kws={'label': 'Процент (%)'},
    linewidths=0.5
)
plt.title('Доли цветообозначение/все прилагательные (на автора)')
plt.xlabel('Автор')
plt.ylabel('Цвет')
plt.tight_layout()

filename = 'color_distribution_heatmap.png'
plt.savefig(filename, dpi=300, bbox_inches='tight')
plt.close()

print(f"Сохранён файл: {filename}")

Сохранён файл: color_distribution_heatmap.png


5.2.1 Облака слов для топа сочетаний цвет + сущ (у отдельного автора, у всех авторов)

In [9]:
# Функция для создания облака слов и его сохранения
def create_wordcloud(word_freq, filename, title):
    # Создаём объект облака слов
    wc = WordCloud(
        width=1200,
        height=800,
        background_color='white',
        colormap='viridis',
        max_words=10,
        prefer_horizontal=1.0,
        font_path=None,
        scale=2
    )

    wc.generate_from_frequencies(word_freq)

    plt.figure(figsize=(10, 6))
    plt.imshow(wc, interpolation='bilinear')
    plt.axis('off')
    plt.title(title, fontsize=16, pad=20)
    plt.tight_layout()

    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"Облако слов сохранено: {filename}")

# Облако для всех авторов (топ‑10 сочетаний)
top_pairs_all = Counter(all_pairs_lemma).most_common(10)
word_freq_all = dict(top_pairs_all)
create_wordcloud(
    word_freq_all,
    'wordcloud_all_authors.png',
    'Топ‑10 сочетаний «цвет + существительное» (все авторы)'
)

#Облака для каждого автора отдельно
for author in authors:
    author_pairs = author_data[author]['pairs']
    top_pairs_author = Counter(author_pairs).most_common(10)
    word_freq_author = dict(top_pairs_author)
    filename = f'wordcloud_{author}.png'
    title = f'Топ‑10 сочетаний «цвет + существительное» ({author})'
    create_wordcloud(word_freq_author, filename, title)


Облако слов сохранено: wordcloud_all_authors.png
Облако слов сохранено: wordcloud_Пелевин.png
Облако слов сохранено: wordcloud_Сорокин.png
Облако слов сохранено: wordcloud_Иванов.png
Облако слов сохранено: wordcloud_Елизаров.png
Облако слов сохранено: wordcloud_Яхина.png
Облако слов сохранено: wordcloud_Прилепин.png
Облако слов сохранено: wordcloud_Водолазкин.png
Облако слов сохранено: wordcloud_Сальников.png
